In [1]:
given_data_size = 5000
total_trial_num = 10
eta = 1e0
t_max = 25
set_seed = 114530

In [2]:
!pip install normflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normflows: filename=normflows-1.7.2-py2.py3-none-any.whl size=86917 sha256=e5e36c7672cf185458d4b0e93672ab71ba0384976c7c0d7306a741cc8f0c9573
  Stored in directory: /root/.cache/pip/wheels/8a/a4/89/3e09f53a561355c45eccfebeffc07a0e34d36a3f41e3ef68a3
Successfully built normflows


In [3]:
# Import required packages
import torch
import numpy as np
import normflows as nf
import os
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
import torch.nn.functional as F
import torch.distributions as TD
import pandas as pd
import seaborn as sb
import torch.nn as nn
import shutil
import gc

In [5]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
sampled_theta_m = nf.distributions.TwoMoons().to(device)

cov_mx = torch.tensor([[1, 0.0], [0.0, 1.]], dtype=torch.double).to(device)

In [6]:
def get_p1_bottom_faster(wgt, x, theta_list):
  num_samples = theta_list.shape[0]
  given_data_size = x.shape[0]

  std_normal = TD.MultivariateNormal(
            torch.zeros(num_samples, 2).to(device),
            cov_mx.unsqueeze(0).repeat(num_samples, 1, 1),
            validate_args=False)
  x_data_rep = x.repeat(1,1,num_samples).reshape(given_data_size,num_samples,-1)
  prob_mx = torch.exp(std_normal.log_prob(theta_list - x_data_rep))
  wgt_rep = wgt.repeat(given_data_size, 1)
  return torch.nanmean(prob_mx*wgt_rep, 1)*given_data_size


def find_L_n_ver4(x, wgt, theta_list):
  num_samples = theta_list.shape[0]
  given_data_size = x.shape[0]
  mu = theta_list
  std_normal = TD.MultivariateNormal(
            mu.to(device),
            cov_mx.to(device).unsqueeze(0).repeat(num_samples, 1, 1).to(device),
            validate_args=False)
  x_data_rep = x.repeat(1,1,num_samples).reshape(given_data_size,num_samples,-1)
  prob_mx = torch.exp(std_normal.log_prob(x_data_rep))
  prob_mx_log_col_mean = torch.log(torch.sum(prob_mx*wgt.repeat(given_data_size,1), dim = 1))
  return -torch.mean(prob_mx_log_col_mean)

In [ ]:
Ln_rho_k_list = np.zeros((total_trial_num, t_max))

for trail_num in range(total_trial_num):

  torch.manual_seed(set_seed + trail_num)

  sampled_mean = sampled_theta_m.sample(given_data_size).to(device).to(torch.float64)
  normal_temp = TD.MultivariateNormal(
        sampled_mean.to(device),
        cov_mx.to(device).unsqueeze(0).repeat(given_data_size, 1, 1).to(device),
        validate_args=False)

  given_data = normal_temp.sample().detach().to(device)

  grid_size = 55
  L = torch.max(given_data).item()
  xx, yy = torch.meshgrid(torch.linspace(-L, L, grid_size), torch.linspace(-L, L, grid_size))
  zz = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)
  zz = zz.to(device)
  num_samples = grid_size ** 2

  mu = zz
  wgt = torch.tensor([1/num_samples], dtype=torch.float64).repeat(num_samples).to(device)
  L_n_loss_list = np.array([])

  folder_name = 'trial' + str(trail_num)
  for t in tqdm(range(t_max)):
    num_samples = grid_size ** 2 # m
    given_data_size = given_data.shape[0]

    std_normal = TD.MultivariateNormal(
              torch.zeros(num_samples, 2).to(device),
              cov_mx.unsqueeze(0).repeat(num_samples, 1, 1),
              validate_args=False)

    x_data_rep = given_data.repeat(1,1,num_samples).reshape(given_data_size,num_samples,-1)

    prob_mx_new = torch.transpose(torch.exp(std_normal.log_prob(mu - x_data_rep)), 0, 1)
    new_bottom = get_p1_bottom_faster(wgt, given_data, mu).repeat(1,1,num_samples).reshape(num_samples,given_data_size)
    wgt_update = torch.nanmean(prob_mx_new/new_bottom,1) - 1
    wgt = wgt + eta*wgt_update*wgt
    wgt = wgt/torch.sum(wgt)

    L_n_loss = find_L_n_ver4(given_data, wgt, mu)
    L_n_loss_list = np.append(L_n_loss_list, L_n_loss.to('cpu').data.numpy())

    if t % 5 == 4:
      print('trail:',trail_num, 'L_n :', L_n_loss.item() )

  Ln_rho_k_list[trail_num,:] = L_n_loss_list

